## Building a RAG to predit risk score of a transaction 

### Initiating model using groq inference server

In [2]:
import getpass
import os

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "Anish@hack1"


from langchain.chat_models import init_chat_model

model = init_chat_model("deepseek-r1-distill-qwen-32b", model_provider="groq")


### Simple prompt with SystemMessage and HumanMessage

In [13]:
# from langchain_core.messages import HumanMessage, SystemMessage

# transaction = input('Enter transaction: ')

# messages = [
#     SystemMessage("You are an assitant that helps data analysts in a financial institution to risk score a transaction among entities/corporations. It can also include individuals. Given an input transaction, you need to predict the risk score (0 to 1) of the transaction, confidence score (0 to 1) and a reason for the risk score."),
#     HumanMessage(transaction),
# ]
## Update this with structured output? To get a JSON output??
messages = [
    SystemMessage("Given an input transaction, extract the entities involved, the value of the transaction, the date of the transaction, the type of transaction, the location and any other information that you think is relevant."),
    HumanMessage(transaction),
]
output = model.invoke(messages)
print(output)

content='<think>\nAlright, so I need to figure out how to extract information from a transaction. Let me start by looking at the given example to understand what\'s expected. The user provided a transaction with various fields like Transaction ID, Date, Payer, etc., and the assistant broke it down into categories: Transaction ID, Date, Payer, Payee, Amount, Payment Method, Purpose, Status, Location, and any additional notes.\n\nOkay, so if I were to approach this, I should first identify all the possible fields that a transaction might have. From the example, it looks like the main fields are ID, Date, Payer, Payee, Amount, Method, Purpose, and Status. But there might be more, like account numbers, banks involved, locations, etc.\n\nI should think about each part of the transaction. Let\'s take the given example. The transaction ID is TXN20250322113045. That\'s straightforward. The date is 2025-03-22T11:30:45Z, which includes the date and time in a specific format. Payer is Tesla, Inc.

In [17]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_template(
    """
    You are an agent whose output is the input for a GraphCyperChainQA that generates a query, queries and gives a output from a GraphDB. The database has relationships among different entities (companies, officers). Using the context generate human readable text that when given as an input to the subsequent chain the chain should generate good and useful responses. Use the context if needed.
    Context: {context}
    """
)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

chain = (
    {"output": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)
# chain = (
#     {"context": retriever, "transaction": RunnablePassthrough()},
#     | prompt
#     | model
#     | StrOutputParser()
# )
chain.invoke()

TypeError: RunnableSequence.invoke() missing 1 required positional argument: 'input'

### Creating a Chain with prompt (chatprompttemplate) and context (wikipedia retriever) and model 

#### Wikipedia Retriever 

In [7]:
from langchain_community.retrievers import WikipediaRetriever

# documentloader 
wiki_retriever = WikipediaRetriever()

In [118]:
# docs = wiki_retriever.invoke("Adani Green Energy")
# for doc in docs:
#     print(doc)
#     print("===")

# print(docs[2].page_content[:400])

#### Chain 

In [119]:
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.runnables import RunnablePassthrough

# prompt = ChatPromptTemplate.from_template(
#     """
#     You are an agent that helps data analysts in a financial institution by risk scoring a transaction 
#     among entities/corporations. It can also include individuals. Given an input transaction, you need to output 
#     the risk score (0 to 1) of the transaction, confidence score (0 to 1). Use the context if needed.
#     Context: {context}
#     Transaction: {transaction}
#     """
# )


# def format_docs(docs):
#     return "\n\n".join(doc.page_content for doc in docs)

# chain = (
#     {"context": wiki_retriever | format_docs, "transaction": RunnablePassthrough()}
#     | prompt
#     | model
#     | StrOutputParser()
# )
# # chain = (
# #     {"context": retriever, "transaction": RunnablePassthrough()},
# #     | prompt
# #     | model
# #     | StrOutputParser()
# # )

In [120]:
# # transaction_test = input()
# chain.invoke(
#     "Transaction ID: TXN20250322113045, Date: 2025-03-22T11:30:45Z,  Payer: Tesla, Inc.  Payer Account Number: TESLA987654321,  Payer Bank: JPMorgan Chase Bank, USA, Payee: Adani Green Energy Ltd.  ,Payee Account Number: ADANIGREEN123456  ,Payee Bank: State Bank of India, India  ,Amount: 500,000,000 USD  ,Payment Method: Wire Transfer  ,Purpose: Investment in Renewable Energy Collaboration  ,Status: Completed  "
# )

In [ ]:
# from langchain_neo4j import Neo4jGraph

# graph = Neo4jGraph()


In [121]:
# graph.refresh_schema()
# print(graph.schema)

In [122]:
# enhanced_graph = Neo4jGraph(enhanced_schema=True)
# print(enhanced_graph.schema)

In [123]:
# ## Note: Sometimes this may return an error. It depends on the how well the llm is able to generate the cypher query. 
# ## Few-shot prompting is one way to improve the performance of the llm.
# from langchain_neo4j import GraphCypherQAChain

# chain = GraphCypherQAChain.from_llm(
#     graph=enhanced_graph, llm=model, verbose=True, allow_dangerous_requests=True, validate_cypher=True
# )
# response = chain.invoke({"query": "Name the officers that are linked with French entities"})
# print(response) # response becomes context 


# ## Input: Transaction -> Process the input to extract entities and generate a prompt to generate a query -> Query the graph database to get the required information and store it -> Pass this along with Wikipedia retriever to generate the risk_score and confidence score 
# ## Should we use separate chains? Is it possible to use one chain to do all of this?

# ## Output: Risk Score, Confidence Score, Reason for the risk score

# ## catching error, timeout ,

In [2]:
import os

api_key = input("Enter your OpenSanctions API Key: ")
os.environ["OPENSANCTIONS_API_KEY"] = api_key

In [125]:
# from pprint import pprint
# import requests
# import os

# OS_API_KEY = os.getenv("OPENSANCTIONS_API_KEY")
# if not OS_API_KEY:
#     raise ValueError("The OS_API_KEY environment variable is not set")

# def fetch_sanctions_data(person_name, company_name):
#     headers = {"Authorization": OS_API_KEY}

#     query = {
#         "queries": {
#             "query-A": {"schema": "Person", "properties": {"name": [person_name]}},
#             "query-B": {"schema": "Company", "properties": {"name": [company_name]}},
#         }
#     }

#     response = requests.post(
#         "https://api.opensanctions.org/match/default", headers=headers, json=query
#     )
#     response.raise_for_status()
#     response_json = response.json()

#     print("\nFull API Response:")
#     pprint(response_json, sort_dicts=False)

#     # Define known risky datasets
#     risky_datasets = {
#         "us_ofac_sdn", "eu_travel_bans", "gb_hmt_sanctions", "ua_nsdc_sanctions",
#         "eu_fsf", "lt_fiu_freezes", "ca_dfatd_sema_sanctions", "jp_mof_sanctions",
#         "ch_seco_sanctions", "ru_acf_bribetakers", "be_fod_sanctions", "au_dfat_sanctions"
#     }
    
#     for query_id, query_response in response_json["responses"].items():
#         print(f"\nResults for query {query_id}:")
#         results = []
        
#         for result in query_response["results"]:
#             entity_info = {
#                 "id": result["id"],
#                 "name": result["properties"].get("name", []),
#                 "match": result["match"],
#                 "topics": result["properties"].get("topics", []),
#                 "datasets": result.get("datasets", []),
#             }
#             results.append(entity_info)

#         pprint(results, sort_dicts=False)

#         # **Improved Risk Detection**
#         risky_entities = []
#         for entity in results:
#             entity_topics = set(entity.get("topics", []))
#             entity_datasets = set(entity.get("datasets", []))

#             if entity_topics.intersection({"sanction", "pep", "crime", "corruption", "terrorism"}) or \
#                entity_datasets.intersection(risky_datasets):
#                 risky_entities.append(entity)

#         if risky_entities:
#             print("\n🚨 Risky Entities Detected:")
#             pprint(risky_entities, sort_dicts=False)
#         else:
#             print("\n✅ No risky entities found.")

# # Example Usage:
# fetch_sanctions_data("Vladimir Putin", "")



# # "name": result["properties"].get("name", []),
#                     # "match": result["match"],
#                     # "topics": list(entity_topics),  
#                     # "datasets": list(entity_datasets),  

In [126]:
# from pprint import pprint
# import requests
# import os

# OS_API_KEY = os.getenv("OPENSANCTIONS_API_KEY")
# if not OS_API_KEY:
#     raise ValueError("The OS_API_KEY environment variable is not set")

# def fetch_sanctions_data(person_name, company_name):
#     headers = {"Authorization": OS_API_KEY}

#     query = {
#         "queries": {
#             "query-A": {"schema": "Person", "properties": {"name": [person_name]}},
#             "query-B": {"schema": "Company", "properties": {"name": [company_name]}},
#         }
#     }

#     response = requests.post(
#         "https://api.opensanctions.org/match/default", headers=headers, json=query
#     )
#     response.raise_for_status()
#     response_json = response.json()

#     print("\nFull API Response:")
#     pprint(response_json, sort_dicts=False)

#     # Define known risky datasets and topics
#     risky_datasets = {
#     "eu_travel_bans", "gb_hmt_sanctions", "ua_nsdc_sanctions", "jp_mof_sanctions",
#     "ca_dfatd_sema_sanctions", "ch_seco_sanctions", "lt_fiu_freezes", "au_dfat_sanctions",
#     "us_ofac_sdn", "ru_acf_bribetakers", "be_fod_sanctions", "eu_fsf",
#     "in_nse_debarred"  # ✅ Added NSE debarred list
# }


#     risky_topics = {"wanted", "sanction", "poi", "corruption", "crime", "role.pep", "terrorism", "reg.warn"}


#     for query_id, query_response in response_json["responses"].items():
#         print(f"\nResults for query {query_id}:")
#         results = []
        
#         for result in query_response["results"]:
#             entity_topics = set(result["properties"].get("topics", []))  # Fix extraction
#             entity_datasets = set(result.get("datasets", []))  # Fix extraction

#             print(f"\n🔎 Checking Entity: {result['id']}")
#             print(f"📌 Topics: {entity_topics}")
#             print(f"📌 Datasets: {entity_datasets}")

#             entity_info = {
#                 "id": result["id"],
#                 "name": result["properties"].get("name", []),
#                 "match": result["match"],
#                 "topics": list(entity_topics),  
#                 "datasets": list(entity_datasets),  
#             }
#             results.append(entity_info)

#         pprint(results, sort_dicts=False)

#         # **Improved Risk Detection**
#         risky_entities = []
#         for entity in results:
#             entity_topics = set(entity["topics"])
#             entity_datasets = set(entity["datasets"])

#             print(f"\n🧐 Checking Risk for {entity['id']}:")
#             print(f"🟡 Entity Topics: {entity_topics}")
#             print(f"🟡 Risky Topics: {risky_topics}")
#             print(f"🔹 Matching Topics: {entity_topics.intersection(risky_topics)}")

#             print(f"🟢 Entity Datasets: {entity_datasets}")
#             print(f"🟢 Risky Datasets: {risky_datasets}")
#             print(f"🔸 Matching Datasets: {entity_datasets.intersection(risky_datasets)}")

#             if entity_topics.intersection(risky_topics) or entity_datasets.intersection(risky_datasets):
#                 risky_entities.append(entity)

#         if risky_entities:
#             print("\n🚨 Risky Entities Detected:")
#             pprint(risky_entities, sort_dicts=False)
#         else:
#             print("\n✅ No risky entities found.")

# # Example Usage:
# fetch_sanctions_data("",  "Adani Green Energy Ltd.")


In [ ]:
#added changes to the branch

In [9]:
import requests
from langchain.schema.document import Document
from langchain.schema.retriever import BaseRetriever

class OpenSanctionsRetriever(BaseRetriever):

    def _get_relevant_documents(self, query):
        """
        Queries OpenSanctions API and returns relevant documents.
        """

        person_name=""
        company_name=""
        api_key = "3b9678eb2e0dff14c268b43f7acf4798"
        if "Company:" in query or "Person:" in query:
            parts = query.split(",")
            for part in parts:
                if "Company:" in part:
                    company_name = part.split("Company:")[-1].strip()
                    print(company_name)
                elif "Person:" in part:
                    person_name = part.split("Person:")[-1].strip()
                    print(person_name)


        headers = {"Authorization": api_key}
        # params = {"q": query}

        query = {
            "queries": {
                "query-A": {"schema": "Person", "properties": {"name": [person_name]}},
                "query-B": {"schema": "Company", "properties": {"name": [company_name]}},
            }
        }
        response = requests.post(
            "https://api.opensanctions.org/match/default", headers=headers, json=query
        )
        # if response.status_code != 200:
        #     return []
        
        response.raise_for_status()
        response_json = response.json()

        # print("\nFull API Response:")
        # pprint(response_json, sort_dicts=False)

        # if not response_json.get("results"):
        #     print("empty list lool")
        #     return []

        documents = []
        # print("outside first for")
        for query_id, query_response in response_json["responses"].items():
            # print(f"\nResults for query {query_id}:")
            # results = []
            
            for result in query_response["results"]:
                # print("in for result")
                entity_topics = set(result["properties"].get("topics", []))  # Fix extraction
                entity_datasets = set(result.get("datasets", []))  # Fix extraction

                # print(f"\n🔎 Checking Entity: {result['id']}")
                # print(f"📌 Topics: {entity_topics}")
                # print(f"📌 Datasets: {entity_datasets}")
                
                name_to_store_page_content=result["properties"].get("name")
                # print("hehe", name_to_store_page_content[0])
                entity_info = {
                    "id": result["id"],
                    "name": result["properties"].get("name", []),
                    "match": result["match"],
                    "topics": list(entity_topics),  
                    "datasets": list(entity_datasets),  
                }
                doc = Document(page_content=f"Sanctions data for {name_to_store_page_content}", metadata=entity_info)
                # print("doc = ", doc)
                documents.append(doc)
                # results.append(entity_info)
        return documents


In [10]:
OpenSanctions_retriever = OpenSanctionsRetriever()
# sanction_docs = OpenSanctions_retriever._get_relevant_documents("Company: Adani Green Energy Ltd., Person: Adani")
# print("sanction_docs = ", sanction_docs)
# # print(sanction_docs)

In [128]:
# for doc in sanction_docs:
#     print(doc)
#     print("===")

In [130]:
# docs = ensemble_retriever.invoke("Company: Adani Green Energy Ltd., Person: Adani")
# docs
# # for doc in docs:
# #     print(doc.metadata)
# #     print(doc.page_content)
# #     print("------")

In [4]:
from langchain.document_loaders import TextLoader
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# Step 1: Load and vectorize Instructions.md
loader = TextLoader("Instructions.md")
docs = loader.load()

# Use a local embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Store Instructions.md in FAISS and create a retriever
vectorstore = FAISS.from_documents(docs, embedding_model)
instructions_retriever = vectorstore.as_retriever()

/var/folders/l7/n743976x54b5sf64vktc7md40000gn/T/ipykernel_17730/999324814.py:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [11]:
from langchain.retrievers import EnsembleRetriever

ensemble_retriever = EnsembleRetriever(
    retrievers=[wiki_retriever, OpenSanctions_retriever, instructions_retriever], weights=[0.25, 0.5, 0.25]
)

In [12]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

prompt_test = ChatPromptTemplate.from_template(
    """
    You are an agent that helps data analysts in a financial institution by risk scoring a transaction 
    among entities/corporations. It can also include individuals. Given an input transaction, you need to output 
    the risk score (0 to 1) of the transaction, confidence score (0 to 1) and reason for your answer.
    Context: {context}
    Transaction: {transaction}
    """
)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

chain_test = (
    {"context": ensemble_retriever | format_docs, "transaction": RunnablePassthrough()}
    | prompt_test
    | model
    | StrOutputParser()
)

In [13]:
chain_test.invoke("TXN20250322113045,2025-03-22T11:30:45Z,Tesla, Inc.,TESLA987654321,JPMorgan Chase Bank, USA,Adani Green Energy Ltd.,ADANIGREEN123456,State Bank of India, India,500000000,USD,Wire Transfer,Investment in Renewable Energy Collaboration,Completed")

'Based on the provided transaction details, let\'s perform the risk scoring analysis:\n\n### **Extracted Key Information**\n\n1. **Key Entity Information:**\n   - **Sender:** Tesla, Inc.\n     - **Entity Type:** Corporation\n     - **Account Number:** TESLA987654321\n   - **Receiver:** Adani Green Energy Ltd.\n     - **Entity Type:** Corporation\n     - **Account Number:** ADANIGREEN123456\n   - **Intermediary Bank:** JPMorgan Chase Bank, USA\n   - **Final Bank:** State Bank of India, India\n\n2. **Transaction Details:**\n   - **Amount:** 500,000,000 USD\n   - **Transaction Type:** Wire Transfer\n   - **Purpose:** Investment in Renewable Energy Collaboration\n   - **Completion Time:** 2025-03-22T11:30:45Z\n\n3. **Additional Risk Indicators:**\n   - **Intermediaries & Routing:** The transaction involves a US bank (JPMorgan Chase) and an Indian bank (State Bank of India).\n   - **Sanctions/Watchlists:** Tesla, Inc. and Adani Green Energy Ltd. need to be checked against sanctions lists.\n